In [5]:
import bs4 as bs
import pandas as pd
import requests
import re

In [6]:
def get_url(x: int) -> str: 
    assert 1 <= x <= 62, "x must be in range 1-62"
    num = str(x)
    if x < 10:
        num = "0"+num
    return "http://oss101.ldd.go.th/web_thaisoilinf/62_soilgroup/62sg_desc/desc_"+num+".html"


get_url(1) == "http://oss101.ldd.go.th/web_thaisoilinf/62_soilgroup/62sg_desc/desc_01.html", get_url(1)

(True,
 'http://oss101.ldd.go.th/web_thaisoilinf/62_soilgroup/62sg_desc/desc_01.html')

In [7]:
def detect_language(text):
    thai_range = (0x0E01, 0x0E5B)  # Thai unicode range
    # English unicode range (upper and lower case)
    english_range = (0x0041, 0x007A)

    langs_dict = {'Thai': 0, 'English': 0, "Numerical": 0, "Unknown": 0}

    # Check if the text contains Thai characters
    for char in text:
        if ord(char) >= thai_range[0] and ord(char) <= thai_range[1]:
            return "Thai"

    # Check if the text contains English characters
    for char in text:
        if (ord(char) >= english_range[0] and ord(char) <= english_range[1]) or (ord(char) >= english_range[0]+32 and ord(char) <= english_range[1]+32):
            return "English"

    if any(char.isdigit() for char in text):
        return "Numerical"

    return "Unknown"


def detect_mix_language(text):
    thai_range = (0x0E01, 0x0E5B)  # Thai unicode range
    # English unicode range (upper and lower case)
    english_range = (0x0041, 0x007A)

    th = False
    en = False

    # Check if the text contains Thai characters
    for char in text:
        if ord(char) >= thai_range[0] and ord(char) <= thai_range[1]:
            th = True
        if (ord(char) >= english_range[0] and ord(char) <= english_range[1]) or (ord(char) >= english_range[0]+32 and ord(char) <= english_range[1]+32):
            en = True
    if th and en:
        return True

    return False


def spliter(string):
    lst = []
    lst_str = "-".join(";".join(string.split(' ')).split(';')).split('-')
    for i in lst_str:
        #         print(detect_mix_language(i), detect_mix_language(string))
        if '(' in i and ')' not in i:
            for j in i.split('('):
                lst.append(j)
        elif '(' in i and detect_mix_language(i) and detect_mix_language(string):

            for j in i.split('('):
                lst.append(j)
        else:
            lst.append(i)
    return lst


def remove_every_unwanted(string):
    string = string.strip()

    lst = []
    for i in string.split(' '):
        #         print(i.endswith(')'))
        if i.startswith('(') and i.endswith(')') and detect_language(i) != 'Thai':
            lst.append(i[(len('(')):len(i)-len(')')])
        elif i.startswith('(') and not i.endswith(')'):
            lst.append(i[(len('(')):])
        elif i.endswith(')') and '(' not in i:
            #             print(i)
            lst.append(i[:len(i)-len(')')])
        else:
            lst.append(i)
    string = " ".join(lst)

    string = string.strip()
    return string

def string2float(text):
    text = text.strip()
    if text == '' or text == ' ' or text == "-":
        return -1
    return float(text)

In [14]:
import bs4 as bs
import requests

lst = []

def extract_useful_info(id:int):
    print(id)
    
    url = get_url(id)
    response = requests.get(url)
    soup = bs.BeautifulSoup(response.content, 'html.parser')

    table = soup.find_all("table", {"width": "100%","border":"0","align":"center","cellpadding":"2","cellspacing":"1"})[0]

    soil_series = table.find_all("td", {"valign": "top", "bgcolor": "#FFF5D7"})[0].text.strip()          # ชุดดินในกลุ่ม
    grp_properties = table.find_all("td", {"valign": "top","bgcolor":"#FFF5D7"})[1].text.strip()         # สมบัติของกลุ่มดิน
    grp_resourcefullness = table.find_all("td", {"valign": "top","bgcolor":"#FFF5D7"})[2].text.strip()   # ความอุดมสมบูรณ์ตามธรรมชาติ
    
    soil_series_id = [remove_every_unwanted(i) for i in spliter(soil_series) if detect_language(i) == 'English']
    
    grp_properties_table = table.find_all("table", {"width": "100%", "border":"1","bordercolor":"#FFFFFF","align":"left","cellpadding":"2","cellspacing":"0"})[0]

    grp_properties_upper_organic = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[1].text.strip()
    grp_properties_upper_N = string2float(grp_properties_upper_organic)*0.05 if grp_properties_upper_organic != '' and grp_properties_upper_organic != "-" else -1
    grp_properties_upper_P = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[2].text.strip()
    grp_properties_upper_K = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[3].text.strip()
    grp_properties_upper_pH = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[4].text.strip()

    grp_properties_lower_organic = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[6].text.strip()
    grp_properties_lower_N = string2float(grp_properties_lower_organic)*0.05 if grp_properties_lower_organic != '' and grp_properties_lower_organic != "-" else -1
    grp_properties_lower_P = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[7].text.strip()
    grp_properties_lower_K = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[8].text.strip()
    grp_properties_lower_pH = grp_properties_table.find_all("td", {"align":"center","valign":"middle","bgcolor":"#FCEEE2"})[9].text.strip()
    
    grp_properties_upper_organic = string2float(grp_properties_upper_organic if grp_properties_upper_organic != '' and grp_properties_upper_organic != "-" else 0) if grp_properties_upper_organic != '' and grp_properties_upper_organic != "-" else None
    grp_properties_upper_P = string2float(grp_properties_upper_P) if grp_properties_upper_P != '' and grp_properties_upper_P != "-" else None
    grp_properties_upper_K = string2float(grp_properties_upper_K) if grp_properties_upper_K != '' and grp_properties_upper_K != "-" else None
    grp_properties_upper_pH = [string2float(i) for i in grp_properties_upper_pH.split('-')]
    grp_properties_upper_pH_upper = grp_properties_upper_pH[1] if len(grp_properties_upper_pH) > 1 else None
    grp_properties_upper_pH_lower = grp_properties_upper_pH[0] if len(grp_properties_upper_pH) > 1 else None
    
    grp_properties_lower_organic = string2float(grp_properties_lower_organic) if grp_properties_lower_organic != '' and grp_properties_lower_organic != "-" else None
    grp_properties_lower_P = string2float(grp_properties_lower_P) if grp_properties_lower_P != '' and grp_properties_lower_P != "-" else None
    grp_properties_lower_K = string2float(grp_properties_lower_K) if grp_properties_lower_K != '' and grp_properties_lower_K != "-" else None
    grp_properties_lower_pH = [string2float(i) for i in grp_properties_lower_pH.split('-')] 
    grp_properties_lower_pH_upper = grp_properties_lower_pH[1] if len(grp_properties_lower_pH) > 1 else None
    grp_properties_lower_pH_lower = grp_properties_lower_pH[0] if len(grp_properties_lower_pH) > 1 else None
    
    grp_awc =  table.find_all("td", {"valign": "top","bgcolor":"#BBD1FF"})[0].text.strip()
    grp_awc_lst = [remove_every_unwanted(i) for i in spliter(
        grp_awc) if detect_language(i) == 'Numerical']
    grp_awc_min = grp_awc_lst[0] if len(grp_awc_lst) > 2 else None
    grp_awc_max = grp_awc_lst[1] if len(grp_awc_lst) > 2 else None
    grp_awc_avg = grp_awc_lst[2] if len(grp_awc_lst) > 2 else None
    
    return [id, soil_series, soil_series_id, grp_properties, grp_resourcefullness, grp_properties_upper_organic, grp_properties_upper_N, grp_properties_upper_P, grp_properties_upper_K, grp_properties_upper_pH, grp_properties_lower_organic, grp_properties_lower_N, grp_properties_lower_P, grp_properties_lower_K, grp_properties_lower_pH, grp_properties_upper_pH_upper, grp_properties_upper_pH_lower, grp_properties_lower_pH_upper, grp_properties_lower_pH_lower, grp_awc, grp_awc_lst, grp_awc_min, grp_awc_max, grp_awc_avg]

for i in range(1, 63):
    lst.append(extract_useful_info(i))

df = pd.DataFrame(lst, columns=["grp_id", "soil_series", "soil_series_id", "grp_properties", "grp_resourcefullness", "grp_properties_upper_organic", "grp_properties_upper_N", "grp_properties_upper_P", "grp_properties_upper_K", "grp_properties_upper_pH", "grp_properties_lower_organic",
                  "grp_properties_lower_N", "grp_properties_lower_P", "grp_properties_lower_K", "grp_properties_lower_pH", "grp_properties_upper_pH_upper", "grp_properties_upper_pH_lower", "grp_properties_lower_pH_upper", "grp_properties_lower_pH_lower", "grp_awc", "grp_awc_lst", "grp_awc_min", "grp_awc_max", "grp_awc_avg"])
df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


,grp_id,soil_series,soil_series_id,grp_properties,grp_resourcefullness,grp_properties_upper_organic,grp_properties_upper_N,grp_properties_upper_P,grp_properties_upper_K,grp_properties_upper_pH,...,grp_properties_lower_pH,grp_properties_upper_pH_upper,grp_properties_upper_pH_lower,grp_properties_lower_pH_upper,grp_properties_lower_pH_lower,grp_awc,grp_awc_lst,grp_awc_min,grp_awc_max,grp_awc_avg
0,1,ชุดดินบ้านหมี่ (Bm) ชุดดินบ้านโภชน์ (Bpo) ชุดด...,"[Bm, Bpo, Br, Ck, Kk, Wa]",เป็นกลุ่มชุดดินที่เกิดจากวัตถุต้นกำเนิดดินพวกต...,ปานกลางถึงสูง,2.2,0.110,18.5,115.8,"[5.5, 6.5]",...,"[6.0, 7.5]",6.5,5.5,7.5,6.0,มีค่าระหว่าง 19.86-22.82 \r\n ...,"[19.86, 22.82, 20.99, 120, 33, 1,500]",19.86,22.82,20.99
1,2,อยุธยา (Ay) ชุดดินบางเขน (Bn) ชุดดินบางน้ำเปรี...,"[Ay, Bn, Bp, Ma, Tq]",เป็นกลุ่มชุดดินที่เกิดจากวัตถุต้นกำเนิดดินพวกต...,ปานกลาง,2.5,0.125,5.0,205.4,"[5.0, 6.0]",...,"[4.5, 5.5]",6.0,5.0,5.5,4.5,มีค่าระหว่าง 20.26-21.03เปอร์เซ็นต์โดยน้ำหนัก ...,"[20.26, 20.51, 120, 33, 1,500]",20.26,20.51,120
2,3,ชุดดินบางกอก (Bk) ชุดดินบางเลน (Bl) ชุดดินบางแ...,"[Bk, Bl, Bph, Cc, Sm]",เป็นกลุ่มชุดดินที่เกิดจากวัตถุต้นกำเนิดดินพวกต...,ปานกลางถึงสูง,2.1,0.105,21.3,195.0,"[5.5, 7.0]",...,"[5.5, 8.0]",7.0,5.5,8.0,5.5,มีค่าระหว่าง 18.62-21.12 \r\n ...,"[18.62, 21.12, 20.07, 120, 33, 1,500]",18.62,21.12,20.07
3,4,ชุดดินบางมูลนาก (Ban) ชุดดินบางปะอิน (Bin) ชุด...,"[Ban, Bin, Cn, Cs, Pm, Rb, Sb, Sin, Ss, Tn, Tr]",เป็นกลุ่มชุดดินที่เกิดจากวัตถุต้นกำเนิดดินพวกต...,ปานกลาง,0.9,0.045,6.7,119.0,"[5.5, 6.5]",...,"[6.5, 8.0]",6.5,5.5,8.0,6.5,มีค่าระหว่าง 20.60-22.66 \r\n ...,"[20.60, 22.66, 21.41, 120, 33, 1,500]",20.60,22.66,21.41
4,5,ชุดดินหางดง (Hd) ชุดดินละงู (Lgu) ชุดดินพาน (P...,"[Hd, Lgu, Ph]",เป็นกลุ่มชุดดินที่เกิดจากวัตถุต้นกำเนิดดินพวกต...,ต่ำ-ปานกลาง,2.0,0.100,10.3,50.2,"[5.5, 6.5]",...,"[6.5, 8.0]",6.5,5.5,8.0,6.5,มีค่าระหว่าง 19.29-21.78 \r\n ...,"[19.29, 21.78, 19.95, 120, 33, 1,500]",19.29,21.78,19.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,58,ชุดดินนราธิวาส \r\n (...,[Nw],เป็นกลุ่มชุดดินที่มีลักษณะคล้ายคลึงกับกลุ่มชุด...,ต่ำ,103.1,5.155,59.5,203.8,"[4.0, 4.5]",...,"[4.0, 4.5]",4.5,4.0,4.5,4.0,ไม่มีข้อมูล,[],None,None,None
58,59,เนื่องจากหน่วยแผนที่นี้เป็นหน่วยผสมของดินหลายช...,"[AC, pd, Alluvial, Complex,, poorly, drained]",กลุ่มดินนี้พบบริเวณที่ราบลุ่มหรือบริเวณพื้นล่า...,ไม่แน่นอนขึ้นอยู่กับชนิดของวัตถุต้นกำเนิดดินใน...,0.4,0.020,2.2,25.0,"[5.5, 7.0]",...,"[-1, -1]",7.0,5.5,-1.0,-1.0,ไม่มีข้อมูล,[],None,None,None
59,60,เนื่องจากหน่วยแผนที่นี้เป็นหน่วยผสมของดินหลายช...,"[AC, wd, Alluvial, Complex,, well, drained]",กลุ่มดินนี้พบบริเวณสันดินริมน้ำ \r\n ...,ไม่แน่นอนขึ้นอยู่กับวัตถุต้นกำเนิดดินในบริเวณน...,2.0,0.100,5.6,59.0,"[6.0, 7.0]",...,"[6.0, 7.0]",7.0,6.0,7.0,6.0,ไม่มีข้อมูล,[],None,None,None
60,61,หน่วยแผนที่นี้ประกอบด้วยดินบริเวณเชิงเขาที่มีด...,[],กลุ่มดินนี้เป็นหน่วยผสมของดินหลายชนิดซึ่งเกิดจ...,ไม่แน่นอน ขึ้นอยู่กับชนิดของวัตถุต้นกำเนิดดินใ...,1.3,0.065,1.5,141.0,"[5.5, 7.0]",...,"[5.0, 6.5]",7.0,5.5,6.5,5.0,ไม่มีข้อมูล,[],None,None,None


['19.86', '22.82', '20.99', '120', '33', '1,500']

In [78]:
df['grp_id'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62], dtype=int64)

In [15]:
df.to_csv("soil_group_data.csv", index=False)
df.to_parquet("soil_group_data.parquet")